In [1]:
import torch
import torch.nn as nn

In [2]:
batch_size = 128 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout_rate = 0.2

In [3]:
with open('tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read() 

In [4]:
print(len(text))

1115393


In [5]:
print(text[:400])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it 


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(len(chars))
print(''.join(chars))

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [7]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda string: [stoi[ch] for ch in string]
decode = lambda list: [itos[i] for i in list]

In [8]:
print(encode('wuzzzup'))
print(decode(encode('wuzzup')))

[61, 59, 64, 64, 64, 59, 54]
['w', 'u', 'z', 'z', 'u', 'p']


In [9]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:100])

torch.Size([1115393])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [10]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
def get_batch(type):
    data = train_data if type == 'train' else val_data
    start_x = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in start_x])
    y = torch.stack([data[i+1 : i+block_size+1] for i in start_x])
    x, y = x.to(device), y.to(device)

    return x,y

xb, yb = get_batch('train')
print(xb)

for b in range(batch_size):
    for i in range(block_size):
        print(xb[b, :i+1], ' - ', yb[b, i])
    break

In [12]:
from torch.nn import functional as F
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__ ()
        self.key = nn.Linear(n_embd, head_size, bias=False) 
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones((block_size, block_size))))

        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        B, T, C = x.shape # BATCH_SIZE, BLOCK_SIZE, EMBD_SIZE
        k = self.key(x)
        q = self.query(x)
        
        attn = q @ k.transpose(-2, -1) * C**-0.5 # (B,T,C) @ (B, C, T) --> (B, T, T)
        attn = attn.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 
        attn = F.softmax(attn, dim=-1)
        attn = self.dropout(attn)

        v = self.value(x)
        out = attn @ v # (B,T,T) @ (B,T,C) --> (B,T,C)
        return out


In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [14]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout_rate),
        )
    def forward(self, x):
        return self.net(x)

In [15]:
# class LayerNorm: # (used to be BatchNorm1d)

#   def __init__(self, dim, eps=1e-5, momentum=0.1):
#     self.eps = eps
#     self.gamma = torch.ones(dim)
#     self.beta = torch.zeros(dim)

#   def __call__(self, x):
#     # calculate the forward pass
#     xmean = x.mean(1, keepdim=True) # batch mean
#     xvar = x.var(1, keepdim=True) # batch variance
#     xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
#     self.out = self.gamma * xhat + self.beta
#     return self.out

#   def parameters(self):
#     return [self.gamma, self.beta]

In [16]:
class Block(nn.Module):
    def __init__(self, n_embd, n_heads):
        super().__init__()
        head_size = n_embd//n_heads
        self.sa = MultiHeadAttention(num_heads=n_heads, head_size=head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:


class Model(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embd_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd=n_embd, n_heads=n_head) for _ in range(n_layer)])
        self.ln = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)


    def forward(self, tokens, targets = None):
        B,T = tokens.shape

        tok_emb = self.token_embd_table(tokens) # (BATCH_SIZE,VOCAB_SIZE,EMBD_SIZE) B,T,C
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln(x)
        logits = self.lm_head(x) # (BATCH_SIZE, EMBD_SIZE, VOCAB_SIZE)

        if targets == None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, tokens, max_len):
        for _ in range(max_len):
            # logits, loss = self(tokens)
            logits, loss = self(tokens[:, -block_size:])
            logits = logits[:, -1, :] # (BATCH_SIZE, EMBD_SIZE)
            probs = F.softmax(logits, dim=-1)
            token_next = torch.multinomial( probs, num_samples=1)
            tokens = torch.cat((tokens, token_next), dim=1) # (BATCH_SIZE, TOKEN_SIZE+1)

        return tokens


In [18]:
m = Model(vocab_size=vocab_size)
m = m.to(device)

In [20]:
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [21]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [22]:
print(device)

cuda


In [ ]:
for iter in range(max_iters):
    print(iter)
    if (iter+1)%eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter+1}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [24]:
a = torch.tensor(encode('hello my'))
print(a.shape)
print(torch.unsqueeze(a,0).shape)
print(torch.tensor(encode('hello my')).shape)
print(torch.zeros((1,1), dtype=torch.long))

torch.Size([8])
torch.Size([1, 8])
torch.Size([8])
tensor([[0]])


In [ ]:
input = torch.tensor(encode('You will'), dtype=torch.long, device=device) 
input = torch.unsqueeze(input, 0)
decoded = decode(m.generate(tokens=input, max_len=200)[0].tolist())
print(''.join(decoded))

You will be killed by thy name?'
O;Till musit my beneface so indeed,
Not by etemish'd, this stumbly hath a grass.

NORTHUMBERLAND:
Raishly laid of Harry for our six descent.

KING RICHARD II:
No, nor a weeds, hear thee, no pr
